In [ ]:
import pandas as pd
import numpy as np
from docx import Document
import os

def calculate_eigenvector(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_eigenvalue = np.max(eigenvalues)
    max_eigenvector = eigenvectors[:, np.argmax(eigenvalues)]
    normalized_vector = max_eigenvector / np.sum(max_eigenvector)
    return max_eigenvalue, normalized_vector.real

def calculate_consistency(matrix, max_eigenvalue):
    n = matrix.shape[0]
    ci = (max_eigenvalue - n) / (n - 1)
    ri_values = {1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49}
    ri = ri_values.get(n, 1.49)
    cr = ci / ri if ri != 0 else 0
    return ci, cr, ri

def geometric_mean(matrices):
    product_matrix = np.ones_like(matrices[0])
    for matrix in matrices:
        product_matrix *= matrix
    return np.power(product_matrix, 1/len(matrices))

# Read the CSV file
file_path = os.path.expanduser('../data/compatibility_devices.csv')
print(f"File path: {file_path}")  # Debugging

try:
    data = pd.read_csv(file_path, header=None)
    print("File read successfully.")
except Exception as e:
    print(f"Error reading file: {e}")
    raise

# Skip non-numeric rows (headers)
numeric_data = data.apply(pd.to_numeric, errors='coerce')
numeric_data = numeric_data.dropna().reset_index(drop=True)

# Parse the CSV into individual matrices
engineer_matrices = {}
num_engineers = numeric_data.shape[0]

for i in range(num_engineers):
    sub_criteria = numeric_data.iloc[i].values
    matrix = np.eye(3)
    matrix[0, 1] = sub_criteria[0]
    matrix[0, 2] = sub_criteria[1]
    matrix[1, 2] = sub_criteria[2]
    matrix[1, 0] = 1 / matrix[0, 1]
    matrix[2, 0] = 1 / matrix[0, 2]
    matrix[2, 1] = 1 / matrix[1, 2]
    engineer_matrices[f'Engineer {i + 1}'] = matrix

consistent_matrices = []
results = []

# Process each matrix
for engineer, matrix in engineer_matrices.items():
    max_eigenvalue, weights = calculate_eigenvector(matrix)
    ci, cr, ri = calculate_consistency(matrix, max_eigenvalue)
    if cr <= 0.1:
        consistent_matrices.append(matrix)
    results.append({
        'engineer': engineer,
        'matrix': matrix,
        'weights': weights,
        'max_eigenvalue': max_eigenvalue,
        'ci': ci,
        'cr': cr,
        'ri': ri
    })

# Aggregate the matrices
if consistent_matrices:
    aggregate_matrix = geometric_mean(consistent_matrices)
    max_eigenvalue, weights = calculate_eigenvector(aggregate_matrix)
    ci, cr, ri = calculate_consistency(aggregate_matrix, max_eigenvalue)
    aggregate_result = {
        'aggregate_matrix': aggregate_matrix,
        'weights': weights,
        'max_eigenvalue': max_eigenvalue,
        'ci': ci,
        'cr': cr,
        'ri': ri
    }

# Create a Word document and write results
doc = Document()
doc.add_heading('Compatibility with Existing Devices and Systems AHP Results', 0)

for result in results:
    doc.add_heading(result['engineer'], level=1)
    doc.add_paragraph("Pairwise Comparison Matrix:\n")
    table = doc.add_table(rows=3, cols=3)
    for i in range(3):
        row_cells = table.rows[i].cells
        for j in range(3):
            row_cells[j].text = f"{result['matrix'][i, j]:.4f}"
    doc.add_paragraph(f"Weights:\n {result['weights']}")
    doc.add_paragraph(f"Max Eigenvalue: {result['max_eigenvalue']:.4f}")
    doc.add_paragraph(f"Consistency Index (CI): {result['ci']:.4f}")
    doc.add_paragraph(f"Consistency Ratio (CR): {result['cr']:.4f}")
    doc.add_paragraph(f"Random Index (RI): {result['ri']:.4f}")
    doc.add_paragraph("\n")

if consistent_matrices:
    doc.add_heading('Aggregate Results', level=1)
    doc.add_paragraph("Aggregate Pairwise Comparison Matrix:\n")
    table = doc.add_table(rows=3, cols=3)
    for i in range(3):
        row_cells = table.rows[i].cells
        for j in range(3):
            row_cells[j].text = f"{aggregate_result['aggregate_matrix'][i, j]:.4f}"
    doc.add_paragraph(f"Aggregate Weights:\n {aggregate_result['weights']}")
    doc.add_paragraph(f"Aggregate Max Eigenvalue: {aggregate_result['max_eigenvalue']:.4f}")
    doc.add_paragraph(f"Aggregate Consistency Index (CI): {aggregate_result['ci']:.4f}")
    doc.add_paragraph(f"Aggregate Consistency Ratio (CR): {aggregate_result['cr']:.4f}")
    doc.add_paragraph(f"Aggregate Random Index (RI): {aggregate_result['ri']:.4f}")

# Save the document
output_path = ('../results/compatibility_devices_results.docx')
print(f"Saving document to: {output_path}")  # Debugging

try:
    doc.save(output_path)
    print("Results written to compatibility_devices_results.docx")
except Exception as e:
    print(f"Error saving document: {e}")
    raise
